In [1]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 1.2 MB/s 


Предсказание подозрительных операций по банковским картам. Датасет https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download 

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split   #to divide dataset in train and test
#Import scikit-learn metrics module for accuracy calculation
from catboost import CatBoostClassifier
from sklearn import metrics


RS = 42  #we will fix random state


In [3]:
# подключаем гугл диск на котором данные
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [4]:
!cp /content/gdrive/'My Drive'/2022projects/Hack_Rostov_may13_15/archive.zip .

In [7]:
!ls

archive.zip  creditcard.csv  gdrive  sample_data


In [6]:
!unzip archive.zip

Archive:  archive.zip
  inflating: creditcard.csv          


In [8]:
data_df = pd.read_csv('creditcard.csv')
data_df.head(2)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0


In [9]:
data_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Time,284807.0,9.481386e+04,47488.145955,0.000000,54201.500000,84692.000000,139320.500000,172792.000000
V1,284807.0,3.918649e-15,1.958696,-56.407510,-0.920373,0.018109,1.315642,2.454930
V2,284807.0,5.682686e-16,1.651309,-72.715728,-0.598550,0.065486,0.803724,22.057729
V3,284807.0,-8.761736e-15,1.516255,-48.325589,-0.890365,0.179846,1.027196,9.382558
V4,284807.0,2.811118e-15,1.415869,-5.683171,-0.848640,-0.019847,0.743341,16.875344
V5,284807.0,-1.552103e-15,1.380247,-113.743307,-0.691597,-0.054336,0.611926,34.801666
V6,284807.0,2.040130e-15,1.332271,-26.160506,-0.768296,-0.274187,0.398565,73.301626
V7,284807.0,-1.698953e-15,1.237094,-43.557242,-0.554076,0.040103,0.570436,120.589494
V8,284807.0,-1.893285e-16,1.194353,-73.216718,-0.208630,0.022358,0.327346,20.007208
V9,284807.0,-3.147640e-15,1.098632,-13.434066,-0.643098,-0.051429,0.597139,15.594995


In [10]:
data_df.shape

(284807, 31)

In [11]:
train_df = data_df[:280_000]
test_df = data_df[280_000:]

In [12]:
data_df.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [13]:
train_df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [14]:
X_features = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']

In [15]:
#DIVIDE DATA TO TRAIN AND TEST
X_train, X_val, y_train, y_val = train_test_split(train_df[X_features], 
                                                                      train_df['Class'],
                                                                      shuffle=True, 
                                                                      test_size=0.3,random_state=RS)


In [18]:
clf = CatBoostClassifier(
    iterations=1000,
    depth = 16,
    learning_rate=0.01,
    loss_function='CrossEntropy',
    task_type='CPU',
    early_stopping_rounds= 90,
    use_best_model= True,
    random_seed=RS,
    verbose= 20
)

In [19]:
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train,eval_set=(X_val,y_val),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6601570	test: 0.6603350	best: 0.6603350 (0)	total: 11.2s	remaining: 3h 6m 41s
20:	learn: 0.2588379	test: 0.2601291	best: 0.2601291 (20)	total: 3m 55s	remaining: 3h 3m 6s
40:	learn: 0.1050209	test: 0.1062582	best: 0.1062582 (40)	total: 7m 20s	remaining: 2h 51m 38s
60:	learn: 0.0497257	test: 0.0507738	best: 0.0507738 (60)	total: 11m 4s	remaining: 2h 50m 34s
80:	learn: 0.0275459	test: 0.0285103	best: 0.0285103 (80)	total: 14m 38s	remaining: 2h 46m 8s
100:	learn: 0.0174347	test: 0.0183754	best: 0.0183754 (100)	total: 18m 11s	remaining: 2h 41m 52s
120:	learn: 0.0122182	test: 0.0131685	best: 0.0131685 (120)	total: 21m 51s	remaining: 2h 38m 45s
140:	learn: 0.0090840	test: 0.0100534	best: 0.0100534 (140)	total: 25m 21s	remaining: 2h 34m 26s
160:	learn: 0.0071749	test: 0.0081744	best: 0.0081744 (160)	total: 28m 50s	remaining: 2h 30m 16s
180:	learn: 0.0057490	test: 0.0067705	best: 0.0067705 (180)	total: 32m 28s	remaining: 2h 26m 57s
200:	learn: 0.0048621	test: 0.0059212	best: 0.00592

In [20]:
y_pred=clf.predict(X_val)
y_pred[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [21]:
feature_importance = sorted(zip(map(lambda x: round(x, 4), 
                                    clf.feature_importances_), 
                                X_features), 
             reverse=True)

for i in range(len(feature_importance)):
  print(feature_importance[i])

(8.6105, 'V2')
(7.0749, 'V1')
(6.7205, 'V17')
(6.0266, 'V26')
(5.784, 'Amount')
(4.4096, 'V4')
(3.7474, 'V25')
(3.6462, 'V13')
(3.5567, 'V12')
(3.4889, 'V8')
(3.4395, 'V20')
(3.1965, 'V9')
(3.1629, 'V10')
(3.1007, 'V6')
(2.8824, 'V22')
(2.8493, 'V21')
(2.8298, 'V28')
(2.7417, 'V15')
(2.5258, 'V11')
(2.5044, 'V27')
(2.4236, 'V14')
(2.4132, 'V24')
(2.3135, 'V16')
(2.3081, 'V7')
(1.9486, 'V3')
(1.7927, 'V23')
(1.6535, 'V18')
(1.4498, 'V19')
(1.3987, 'V5')


In [22]:
clf.score(X_val,y_val)

0.9996428571428572

In [23]:
y_test = clf.predict(test_df[X_features])
clf.score(y_test,test_df['Class'])

0.9991678801747451

In [24]:
metrics.f1_score(y_val,y_pred)

0.879032258064516

In [25]:
#построим матрицу сопряженности confusion matrix
target_names = ['fraud operation 1', 'normal operation 0']
report = metrics.classification_report(y_val, y_pred,target_names=target_names)
report.split('\n')
print(report[:170])

                    precision    recall  f1-score   support

 fraud operation 1       1.00      1.00      1.00     83864
normal operation 0       0.97      0.80      0.88


ошибки первого рода - пропуск цели

ошибки второго рода - ложное срабатывание

In [26]:
#save model in order to use in Java spring boot application
clf.save_model('clf_cb.cbm',
               format='cbm')

In [27]:
!ls

archive.zip  catboost_info  clf_cb.cbm	creditcard.csv	gdrive	sample_data


In [28]:
#load binary file, we will use in web application
from google.colab import files
files.download('clf_cb.cbm')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>